# Pre-Processing for GRASS GIS

Here, we will pre-process the files downloaded in steps 3 and 4. The primary objective is to re-project files before manipulating them with GRASS GIS in step 6.

<div class="alert alert-block alert-danger">
<b> Project Name Setting: </b> Please set the Project Name for your project </div>

In [ ]:
# CHANGE PROJECT NAME HERE
PROJECT_NAME = 'SERC'

# CURRENT DIRECTORY HERE, SET WHERE THE NOTEBOOKS ARE LOCATED
CURRENT_DIRECTORY = ''

# SET THE SPATIAL RESOLUTION (in meters)
SPATIAL_RESOLUTION = 30

In [ ]:
import os, shutil
os.chdir(CURRENT_DIRECTORY)

In [ ]:
# Define folder paths as global variables
PROJECT_DIR = os.path.join(CURRENT_DIRECTORY, PROJECT_NAME)
RAWGIS_DIR = os.path.join(PROJECT_DIR, "gis_data")
RAWOBS_DIR = os.path.join(PROJECT_DIR, "obs")
RAWSOIL_DIR = os.path.join(RAWGIS_DIR, "soil")
MODEL_DIR = os.path.join(PROJECT_DIR, 'model')
DEF_DIR = os.path.join(MODEL_DIR, 'defs')
INI_DIR = os.path.join(MODEL_DIR, 'ini_files')
EPC_DIR = os.path.join(MODEL_DIR, 'epc_files')
OUTPUT_DIR = os.path.join(MODEL_DIR, 'output')
CO2_DIR = os.path.join(MODEL_DIR, 'co2')
NDEP_DIR = os.path.join(MODEL_DIR, 'ndep')
ENDPOINT_DIR = os.path.join(MODEL_DIR, 'endpoint_files')
SPINUP_DIR = os.path.join(MODEL_DIR, 'spinup')
NORMAL_DIR = os.path.join(MODEL_DIR, 'normal')
MODEL_RAST_DIR = os.path.join(MODEL_DIR, 'raster_inputs')
IMAGE = os.path.join(PROJECT_DIR, 'image_map')

In [ ]:
# Import necessary packages
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import pyproj
from pyproj import CRS, Transformer, Proj, transform
import datetime as dt 
import xarray as xr
import rioxarray
import shapely
import rasterio
from rasterio.features import Affine
import time
from shapely.geometry import mapping
from netCDF4 import Dataset
import pandas as pd

In [ ]:
# Define the daymet projection
daymet_proj = "+proj=lcc +ellps=WGS84 +a=6378137 +b=6356752.314245 +lat_1=25 +lat_2=60 +lon_0=-100 +lat_0=42.5 +x_0=0 +y_0=0 +units=m +no_defs"

In [ ]:
# Open meteorological data downloaded in Step_1_C
opened = xr.open_dataset(os.path.join(MODEL_DIR, 'met_data', 'aoi_met.nc'))
display(opened)

In [ ]:
# Import GIS files
dem = xr.open_rasterio(os.path.join(RAWGIS_DIR, 'nasa_dem.tif'))
nlcd = xr.open_rasterio(os.path.join(RAWGIS_DIR, 'nlcd.tif'))

display(dem)
display(nlcd)

##  Reproject GIS Files

We need to reproject the GIS files (DEM, land use, soils) downloaded from Geoserver to the same projection as the Daymet data. An easy way to do this is use **rioxarray's** reproject function.

In [ ]:
# Reproject GIS files to Daymet projection

dem_reprojected = dem.rio.reproject(daymet_proj)
display(dem_reprojected)

nlcd_2 = nlcd.rio.write_nodata(0)
nlcd_reprojected = nlcd_2.rio.reproject(daymet_proj)
display(nlcd_reprojected)

In [ ]:
# Save re-projected GIS files
dem_reprojected.rio.to_raster(os.path.join(RAWGIS_DIR, 'dem_reprojected.tif'))
nlcd_reprojected.rio.to_raster(os.path.join(RAWGIS_DIR, 'nlcd_reprojected.tif'))

##  Create Text Raster of Daymet Cell IDs

Generate an raster file corresponding to the appropriate Daymet cell to use. Uses an ID system which uniquely number each Daymet cell for easy parallelization later.

In [ ]:
# Pre-processing for lat/lon file conversion
unique_both = []
count = 0
for i in range(len(opened['y'])):
    x_len = len(opened['x'])
    
    row_list = list(range(x_len*count, x_len*(count+1)))
    unique_both.append(row_list)
    
    count = count+1

opened = opened.assign(cell_num=(['y', 'x'], unique_both)).astype(float)
display(opened)

In [ ]:
# Display Daymet cell number GIS file
cell_num = opened['cell_num']
cell_num = cell_num.reset_coords(names=['lat', 'lon'], drop=True)
#cell_num = cell_num.rio.set_spatial_dims('x', 'y')
cell_num = cell_num.rio.set_crs(daymet_proj)
display(cell_num)

In [ ]:
# Create Daymet cell reference file from GIS file

x_coords = cell_num['x'].values
y_coords = cell_num['y'].values

cell_reference = []

row_num = 0
for row in cell_num.values:
    column_num = 0
    for column in row:
        df_row = [int(column), x_coords[column_num], y_coords[row_num]]
        cell_reference.append(df_row)
        column_num += 1
    row_num += 1
    
cell_df = pd.DataFrame(cell_reference)
cell_df.to_csv(os.path.join(DEF_DIR, 'met_cells.csv'), index=False, header=False)

In [ ]:
# Display resampled daymet cell identifier
cell_num_final = cell_num.rio.reproject_match(dem_reprojected)
display(cell_num_final)

In [ ]:
# Save resampled Daymet Cell ID file
cell_num_final.rio.to_raster(os.path.join(RAWGIS_DIR, 'daymet_cell.tif'))

##  Reproject SOIL

In [ ]:
# Main Soil File
soil_spatial = gpd.read_file(os.path.join(RAWSOIL_DIR, 'spatial', 'soilmu_a_aoi.shp'))

In [ ]:
# Re-project AOI
daymet_crs = pyproj.CRS(daymet_proj)
soil_reprojected  = soil_spatial.to_crs(daymet_crs)

In [ ]:
# Display Reprojected AOI
soil_reprojected.plot()
plt.show()

In [ ]:
# Save Reprojected AOI
soil_reprojected.to_file(os.path.join(RAWGIS_DIR, "soil_reprojected.shp"))

##  Reproject AOI

We need to reproject the AOI shapefile created in Step 1_A to the same projection as the Daymet data.

In [ ]:
# Open AOI
aoi = gpd.read_file(os.path.join(RAWGIS_DIR, 'AOI.shp'))

In [ ]:
# Re-project AOI
daymet_crs = pyproj.CRS(daymet_proj)
aoi_reprojected  = aoi.to_crs(daymet_crs)

In [ ]:
# Display Reprojected AOI
aoi_reprojected.plot()

In [ ]:
# Save Reprojected AOI
aoi_reprojected.to_file(os.path.join(RAWGIS_DIR, "AOI_reprojected.shp"))